In [1]:
import pandas as pd

In [7]:
import glob
import os

# Find all CSV files in the data/output directory that match the expected pattern
csv_files = glob.glob("../data/output/synthetic_emails_*.csv")

if not csv_files:
    raise FileNotFoundError("No synthetic_emails CSV files found in data/output/")

# Extract the period (e.g., jan2025_jun2025) from the filenames and sort to get the latest
def extract_period(filename):
    # filename: data/output/synthetic_emails_jan2025_jun2025.csv
    base = os.path.basename(filename)
    parts = base.replace(".csv", "").split("_")
    # period is everything after 'synthetic_emails_'
    return "_".join(parts[2:])

# Sort files by period (lexicographically, which works for this format)
csv_files_sorted = sorted(csv_files, key=extract_period)
latest_csv = csv_files_sorted[-1]

print(f"Loading latest period data from: {latest_csv}")
df_latest = pd.read_csv(latest_csv)


Loading latest period data from: ../data/output/synthetic_emails_jan2025_jun2025.csv


In [8]:
df_latest.head()

,date,category,subject,body
0,2025-01-12,Admin/Coordination,Request for Scheduling Support with Conference...,"Dear Facilities Team,\n\nI hope this email fin..."
1,2025-01-19,Product/Stocking Requests,Restocking Request for High-End Scented Candle...,"Dear Wholesale Representative,\n\nI hope this ..."
2,2025-01-28,General Follow-ups,Update on the Digital Transformation Project f...,"Dear Alex Chen,\n\nI hope this email finds you..."
3,2025-01-29,Billing/Invoices,Payment Terms Clarification Regarding Last Inv...,"Dear [Billing Team],\n\nI hope this email find..."
4,2025-01-27,Product/Stocking Requests,Request for Availability of New Seasonal Flavo...,"Dear [Wholesale Manager's Name],\n\nI hope thi..."


In [1]:
# need to remove the instances from databse where the tag is Wander Bear Straight Black Organic Cold Brew Coffee
import sqlite3
import json

db_path = '../data/email_analysis.db'
tag_to_remove = "Wander Bear Straight Black Organic Cold Brew Coffee"

# Connect to the database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Find rows where the tag is present in the tags list
select_query = "SELECT id, tags FROM email_analysis"
rows = cursor.execute(select_query).fetchall()

# Find IDs to delete
ids_to_delete = []
for row in rows:
    row_id, tags_json = row
    try:
        tags = json.loads(tags_json)
    except Exception:
        tags = []
    if tag_to_remove in tags:
        ids_to_delete.append(row_id)

print(f"Number of rows to be deleted: {len(ids_to_delete)}")

# Delete those rows
if ids_to_delete:
    cursor.executemany("DELETE FROM email_analysis WHERE id = ?", [(i,) for i in ids_to_delete])
    conn.commit()
    print(f"Deleted {len(ids_to_delete)} rows.")
else:
    print("No rows found with the specified tag.")

conn.close()

Number of rows to be deleted: 30
Deleted 30 rows.
